## Memory Based Collaborative Filtering
Author: Kunqi Jiang  
2019.3.5  
@MIT LICENSE

In [1]:
import numpy as np
user_item_data = np.array([[5,5,np.nan,0,0],[5,np.nan,4,0,0],[0,np.nan,0,5,5],[0,0,np.nan,4,np.nan]])
print(user_item_data)

[[  5.   5.  nan   0.   0.]
 [  5.  nan   4.   0.   0.]
 [  0.  nan   0.   5.   5.]
 [  0.   0.  nan   4.  nan]]


In [25]:
# calculating mean square error
def evaluate(test_data):
    nan_index = np.isnan(test_data)
    test_data[nan_index] = 0
    return np.sum(test_data*test_data)

ita = 0.01
lambd = 0.1
iteration = 5000
user_num = user_item_data.shape[0]
item_num = user_item_data.shape[1]
feat_num = 2
# random initialize
users = np.random.rand(user_num,feat_num)
items = np.random.rand(item_num,feat_num)

### Collaborative batch training

In [14]:
pre_mse = float('inf')
for j in range(iteration):
    for i in range(user_num):
        pred = items @ users[i]
        truth = user_item_data[i]
        nan_index = np.isnan(truth)
        error = truth - pred
        error[nan_index] = 0
        g = error.reshape(-1,1) * items # boardcast product
        gradient = np.sum(g,0) # batch update
        regularizor = users[i]
        users[i] += ita * (gradient - lambd * regularizor)
    
    for i in range(item_num):
        pred = users @ items[i]
        truth = user_item_data[0:,i]
        nan_index = np.isnan(truth)
        error = truth - pred
        error[nan_index] = 0
        g = error.reshape(-1,1) * users
        gradient = np.sum(g,0)
        regularizor = items[i]
        items[i] += ita * (gradient - lambd * regularizor)
    
    pred = users @ items.T
    diff = user_item_data - pred
    mse = evaluate(diff)
    print("mean square error", mse)
    if (pre_mse - mse < 10e-3):
        break
    pre_mse = mse

mean square error 140.008677638
mean square error 136.629274591
mean square error 133.109864297
mean square error 129.450864982
mean square error 125.662791526
mean square error 121.765908445
mean square error 117.789239543
mean square error 113.768817106
mean square error 109.745170207
mean square error 105.760196845
mean square error 101.853709793
mean square error 98.0600532205
mean square error 94.405221399
mean square error 90.9048537529
mean square error 87.5633412523
mean square error 84.3740926431
mean square error 81.3208251423
mean square error 78.3796089189
mean square error 75.5213343734
mean square error 72.7142863483
mean square error 69.9265789306
mean square error 67.1282973643
mean square error 64.2932807169
mean square error 61.4005411154
mean square error 58.4353455103
mean square error 55.3899871436
mean square error 52.2642556623
mean square error 49.0655892812
mean square error 45.8088715507
mean square error 42.5158290196
mean square error 39.2140006081
mean squa

In [15]:
# user embedding vectors
np.round(users,3)

array([[ 0.136,  2.353],
       [ 0.145,  2.35 ],
       [ 2.399, -0.089],
       [ 1.791, -0.084]])

In [16]:
# item embedding vectors
np.round(items,3)

array([[ 0.068,  2.114],
       [ 0.139,  2.065],
       [ 0.098,  1.63 ],
       [ 2.123, -0.128],
       [ 1.991, -0.116]])

In [17]:
user_item_data 

array([[  5.,   5.,  nan,   0.,   0.],
       [  5.,  nan,   4.,   0.,   0.],
       [  0.,  nan,   0.,   5.,   5.],
       [  0.,   0.,  nan,   4.,  nan]])

In [19]:
# predicted rating
np.round(pred,2)

array([[ 4.98,  4.88,  3.85, -0.01, -0.  ],
       [ 4.98,  4.87,  3.85,  0.01,  0.02],
       [-0.03,  0.15,  0.09,  5.1 ,  4.79],
       [-0.06,  0.08,  0.04,  3.81,  3.57]])

### Collaborative online training

In [26]:
user_non = [[0,1,3,4],[0,2,3,4],[0,2,3,4],[0,1,3]]
item_non = [[0,1,2,3],[0,3],[1,2],[0,1,2,3],[0,1,2]]

for n in range(iteration):
    mse_1 = 0
    for i in range(user_num):
        #g = np.zeros_like(users[i])
        for j in user_non[i]:
            pred = users[i] @ items[j]
            error = user_item_data[i][j] - pred
            mse_1 += error*error
            g = error * items[j]
            # L2 regularize
            users[i] += ita * ( g - lambd * users[i])
    print("mse1",mse_1)
    
    mse_2 = 0
    for i in range(item_num):
        #g = np.zeros_like(items[i])
        for j in item_non[i]:
            pred = items[i] @ users[j]
            error = user_item_data[j][i] - pred
            mse_2 += error*error
            g = error * users[j]
            items[i] += ita * ( g - lambd * items[i])
    print("mse2",mse_2)
    
    if(abs(mse_1 - mse_2) < 10e-4):
        break

mse1 132.699472071
mse2 131.785463186
mse1 129.032914847
mse2 128.065284179
mse1 125.291311403
mse2 124.282367332
mse1 121.501303792
mse2 120.467936602
mse1 117.69815379
mse2 116.660830845
mse1 113.923514455
mse2 112.904925461
mse1 110.22254811
mse2 109.245988909
mse1 106.640603881
mse2 105.728253164
mse1 103.219789255
mse2 102.391074122
mse1 99.9958346626
mse2 99.2660838828
mse1 96.9956336935
mse2 96.3751775981
mse1 94.2357418023
mse2 93.729543284
mse1 91.7219561287
mse2 91.329767477
mse1 89.4499196982
mse2 89.1668780921
mse1 87.4065402982
mse2 87.2240597566
mse1 85.5719214382
mse2 85.4787198673
mse1 83.9214822824
mse2 83.9045965627
mse1 82.4279857897
mse2 82.4736649447
mse1 81.0632758012
mse2 81.1576884868
mse1 79.7996172466
mse2 79.9293531789
mse1 78.6106169606
mse2 78.7629950444
mse1 77.4717633047
mse2 77.6349795952
mse1 76.3606577439
mse2 76.5238150895
mse1 75.2570241817
mse2 75.4100855137
mse1 74.1425787582
mse2 74.2762809641
mse1 73.0008308019
mse2 73.1065888551
mse1 71.81687020

In [27]:
np.round(users,3)

array([[ 0.28 ,  2.286],
       [ 0.304,  2.273],
       [ 2.348, -0.268],
       [ 1.728, -0.209]])

In [28]:
np.round(items,3)

array([[ 0.228,  2.086],
       [ 0.28 ,  2.047],
       [ 0.218,  1.597],
       [ 2.077, -0.266],
       [ 1.951, -0.247]])

In [29]:
pred = users @ items.T
np.round(pred,2)

array([[ 4.83,  4.76,  3.71, -0.03, -0.02],
       [ 4.81,  4.74,  3.7 ,  0.03,  0.03],
       [-0.02,  0.11,  0.08,  4.95,  4.65],
       [-0.04,  0.06,  0.04,  3.64,  3.42]])